# Test Code: Runs Recruitment Monitoring Bot

In [94]:
email = 'sebastian.deri@gmail.com'
password = 'Sociologytemporary1!'
study = 'Pilot'
recruits = 20
worlds = 1

In [100]:
driver = login_site(email, password, study, worlds, recruits)
monitor_site(driver)

You are about to monitor a study with the following parameters:
  1 worlds
  20 recruit(s) per world


Are the above parameters correct? Enter Y/N: y


Continuing on to monitor the recruitment website...
Logging into TurkPrime...
Navigating to Dashboard...
Constructing 2 HIT-class objects for 1 worlds' parties...
  Constructed Pilot1D
  Constructed Pilot1R
  Note: the list 'hits' contains all 2 HIT-class objects printed above
Begin monitoring cycle 0 of recruitment Dashboard...
Preparing to launch HITs...
launchlist0 and launchlist1 complete and added to launchlists
Launching HITs...
  Waiting 19 minutes for HITs to complete...


KeyboardInterrupt: 

# Functions: Main and Helper

In [96]:
"""
Use monitor_site() to monitor TurkPrime recruitment.
Helper funtions: get_surveyid, add_recruit
"""

#load dependencies
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
import random

In [97]:
#main functions
def login_site(email, password, study, worlds, recruits):
    """
    The function lgoin_site() has 5 parameters:
        1) email (TurkPrime email)
        2) password (TurkPrime password)
        3) study (root name of studies' HITs to monitor)
        4) worlds (number of experimental worlds)
        5) recruits (# total recruits per world, with 1:1 ratio per party),
        
    Running monitor_site('usr@exp.com', "mypw", 'Experiment', 20, 1) will
        1) log into TurkPrime with "usr@exp.com" and "mypw"
        2) monitor studies with 'Experiment' root name (e.g., Experiment1D),
        3) recruit 20 people to 1 experimental world (10 Dems, 10 Reps),
    """
    print("You are about to monitor a study with the following parameters:")
    print("  {} worlds".format(worlds))
    print("  {} recruit(s) per world".format(recruits))
    progress = str(input("Are the above parameters correct? Enter Y/N:")).lower()
    while True:
        if progress == 'y':
            print("Continuing on to monitor the recruitment website...")
            break
        else:
            raise RuntimeError("Breaking: check function parameters or input")

    #Create new Chrome session to access TurkPrime
    url_TP = "https://account.turkprime.com/Account/Login"
    driver = webdriver.Chrome("/usr/local/bin/chromedriver") #Mac location
    driver.implicitly_wait(10) #if load fails in 10s, give error
    driver.get(url_TP)
    time.sleep(5) #wait for TP login page load

    #Log into TP with username and pw
    try:
        print("Logging into TurkPrime...")
        inputRemem = driver.find_element_by_xpath('/html/body/div[1]/div/div/div/form[1]/div[3]/div[3]/div[1]/label').click()
        inputEmail = driver.find_element_by_xpath('//*[@id="Email"]')
        inputEmail.send_keys(email)
        inputPass = driver.find_element_by_xpath('//*[@id="Password"]')
        inputPass.send_keys(password)
        inputPass.send_keys(Keys.ENTER)
        time.sleep(5) #wait after login
    except:
        raise RuntimeError("  Bot error: could not log in")

    #Navigate to Dashboard page and display 20 studies
    try:
        print("Navigating to Dashboard...")
        url_Dash = 'https://www.turkprime.com/LaunchedSurvey/Dashboard?page=1&filter=All&itemsPerPage=20'
        driver.get(url_Dash)
        time.sleep(5)
    except:
        raise RuntimeError("  Bot error: could not navigate to Dashboard")
    
    return driver


def monitor_site(driver):
    """
    This function takes the driver created by login_site and monitors the site using
      the driver created by the login function.
    """
    #Initialize HIT-classes
    hits = [] #holds all HIT-class objects
    print("Constructing {} HIT-class objects for {} worlds' parties...".format(str(worlds*2),str(worlds)))
    index = 1 #initialize index of first position of HIT on Dashboard
    for world in range(1,worlds+1):
        globals()[study+'{}D'.format(world)] = hitGroup(study+str(world)+'D', index, 0, recruits/2)
        hits.append(globals()[study+'{}D'.format(world)])
        print("  Constructed {}".format(globals()[study+'{}D'.format(world)].name))
        index += 2 #increment in odds to match Dashboard's indexing pattern for HITs
        globals()[study+'{}R'.format(world)] = hitGroup(study+str(world)+'R', index, 0, recruits/2)
        hits.append(globals()[study+'{}R'.format(world)])
        print("  Constructed {}".format(globals()[study+'{}R'.format(world)].name))
        index += 2
    print("  Note: the list 'hits' contains all {} HIT-class objects printed above".format(str(len(hits))))
    
    
    #Monitor recruitment from Dashboard
    monitorCycle = 0
    print("Begin monitoring cycle {} of recruitment Dashboard...".format(str(monitorCycle)))

    #Prepare HIT launch lists by random sampling
    print("Preparing to launch HITs...")
    launchlist0 = []
    launchlist1 = []
    i = 0
    while i < len(hits):
        launch0,launch1 = random.sample([hits[i],hits[i+1]],2)
        launchlist0.append(launch0)
        launchlist1.append(launch1)
        i += 2
    launchlists = [launchlist0,launchlist1]
    print("launchlist0 and launchlist1 complete and added to launchlists")

    for launchlist in launchlists:
        #Launch HITs
        print("Launching HITs...")
        for hit in launchlist:
            hit.click_launch()
            time.sleep(5)
        #Wait for launched HITs to complete
        m = 1
        while m < 20:
            print("  Waiting {} minutes for HITs to complete...".format(str(20-m)))
            time.sleep(60)
            url_Dash = 'https://www.turkprime.com/LaunchedSurvey/Dashboard?page=1&filter=All&itemsPerPage=20'
            m += 1
        #Pause launched HITs
        print("Pausing HITs from {}".format(str(launchlist)))
        for hit in launchlist:
            hit.click_pause()
            time.sleep(5)

    monitorCycle += 1
    print("Begin monitoring cycle {} of recruitment Dashboard...".format(str(monitorCycle)))
            
    #Prepare turn-taking order list to balance launchlist
    order0 = [1,1,3,3,5,5,7,7,9,9,10]
    order1 = [0,2,2,4,4,6,6,8,8,10,10]
    order = list(zip(order0,order1))
    launchlist1TF = []
    i = 0
    while i < len(order):
        launchlist1TF.append(order[i][0] > order[i][1])
        i += 1

    #Run resume_HITs() twice per launchlist to keep order in order list
    for e in launchlist1TF:
        if e == True:
            monitorCycle += 1
            print("Begin monitoring cycle {} of recruitment Dashboard...".format(str(monitorCycle)))
            try: resume_HITs(launchlist1)
            except: raise RuntimeError("  resume_HITs() failed")
            monitorCycle += 1
            print("Begin monitoring cycle {} of recruitment Dashboard...".format(str(monitorCycle)))
            try: resume_HITs(launchlist1)
            except: raise RuntimeError("  resume_HITs() failed")
        if e == False:
            monitorCycle += 1
            print("Begin monitoring cycle {} of recruitment Dashboard...".format(str(monitorCycle)))
            try: resume_HITs(launchlist0)
            except: raise RuntimeError("  resume_HITs() failed")
            monitorCycle += 1
            print("Begin monitoring cycle {} of recruitment Dashboard...".format(str(monitorCycle)))
            try: resume_HITs(launchlist0)
            except: raise RuntimeError("  resume_HITs() failed")


    #End monitoring
    print("\n\n\n\n")
    print("************************************************************")
    print("************************************************************")
    print("RECRUITMENT MONITORING ENDED")
    print("Please ensure TurkPrime is in good order before logging off.")
    closeBrowser = str(input("Do you wish to close the bot's browser? Enter Y/N: ")).lower()
    if closeBrowser == "y":
        print("Closing the bot's browser.")
        driver.quit()
    else:
        print("Keeping the bot's browser open.")
        pass
    print("\n\n")
    print("This recruitment monitoring program is now complete.")
    print("The recruitment website will no longer be monitored.")
    print("Goodbye!\n\n")

In [98]:
#helper functions: resume_HITs, compare_HITs, add_recruit

def resume_HITs(launchlist):
    """
    Resumes a launchlist of HITs, waits for their completion, then pauses them.
    """
    global driver
    
    print("Resuming HITs...")
    for hit in launchlist:
        hit.click_resume()
        time.sleep(5)
    #Wait for launched HITs to complete
    m = 1
    while m < 20:
        print("  Waiting {} minutes for HITs to complete".format(str(20-m)))
        time.sleep(60)
        url_Dash = 'https://www.turkprime.com/LaunchedSurvey/Dashboard?page=1&filter=All&itemsPerPage=20'
        m += 1
    #Pause launched HITs
    print("Pausing HITs...")
    for hit in launchlist:
        hit.click_pause()
        time.sleep(5)

def compare_HITs(hits):
    """
    Compare parties' HIT recruitment equality across each world
    """
    global driver
    
    HITsToUpdate = []
    try:
        print("Refreshing the Dashboard...")
        driver.get(url_Dash)
        time.sleep(3)
    except:
        raise RuntimeError("  Bot error: could not reload dashboard")
    try:
        print("Testing parties' HIT recruitment equality across each world...")
        hitcount = len(hits)
        hit = 0
        while hit <= hitcount-1:
            result = hits[hit].compare_progress(hits[hit+1])
            if str(result) == "Equal":
                pass
            if str(result) != "Equal":
                HITsToUpdate.append(result)
            hit += 2
    except:
        raise RuntimeError("  Bot error: HIT recruitment equlity testing failed")
    print("Returned {} HIT-class object(s) in need of updated recruitment counts".format(str(len(HITsToUpdate))))
    return HITsToUpdate


def add_recruit(HITposition, number):
    """
    Increment a HIT's recruitment count by the number parameter
    """
    global driver
    
    try:
        print("Editing HIT {}: addding recruit...".format(str(HITposition)))
        surveyid = get_surveyid(HITposition)
        url_HIT_edit = 'https://www.turkprime.com/LaunchedSurvey/EditPanelStudyToolkit/'+surveyid
        driver.get(url_HIT_edit)   
        time.sleep(5)
        #driver.find_element_by_xpath('//*[@id="page-top"]/div[6]/div/div[10]/button[1]').click() #edit warning
        #time.sleep(5)
    except:
        raise RuntimeError("  Bot error: could not edit HIT") 
    try:
        print("Updating HIT {} recruit count...".format(str(HITposition)))
        recruitsCurrent = driver.find_element_by_id("LaunchedSurveySettings_ParticipantsCount")
        recruitsCurrentN = recruitsCurrent.get_attribute("value")
        print("  HIT {} presently has {} recruits".format(str(HITposition),recruitsCurrentN))
        recruitsUpdatedN = str(int(recruitsCurrentN) + number)
        recruitsCurrent.clear()
        recruitsCurrent.send_keys(recruitsUpdatedN)
        print("  HIT {} updated to have {} recruits".format(str(HITposition),recruitsUpdatedN))
        time.sleep(3)
    except:
        raise RuntimeError("  Bot error: could not add another recruit")
    try:
        print("Navigating to HIT {} save page...".format(str(HITposition)))
        driver.find_element_by_xpath('//*[@id="design-wizard-t-8"]').click()
        time.sleep(3)
    except:
        raise RuntimeError("  Bot error: could not navigate to save HIT page") 
    try:
        print("Saving HIT {}...".format(str(HITposition)))
        driver.find_element_by_xpath('//*[@id="submitBtn"]').click()
        time.sleep(3)
    except:
        raise RuntimeError("  Bot error: could not save HIT") 
    print("HIT {} sucessfully updated!".format(str(HITposition)))
    print("************************************************************\n")

# Class Constructor for Parties' HITs across Worlds

In [102]:
class hitGroup:
    """
    Class constructor: group of recruits with same party, HIT, and world.
    """
    
    global driver
    
    def __init__(self, groupName, groupIndex, completed, remaining):
        self.name      = groupName #project title for group name on TurkPrime
        self.index     = groupIndex #study group index position on Dashboard
        self.completed = completed #how many recruits completed the test so far
        self.remaining = remaining #how many recruits remain to be recruited
        self.root      = '//*[@id="surveyTableBody"]/tr['+str(groupIndex)
        self.project   = driver.find_element_by_xpath(self.root+']/td[1]/div/ul/li[1]/span/a').text
        self.surveyID  = driver.find_element_by_xpath(self.root+']/td[1]/div/ul/li[3]/span')
        self.surveyID  = self.surveyID.text[self.surveyID.text.index(":")+2:] #slice off id
        self.status    = driver.find_element_by_xpath(self.root+']/td[2]/span').text
        self.progress  = driver.find_element_by_xpath(self.root+']/td[3]/div[1]/div/p').text
        self.done      = self.progress[:self.progress.index("/")-1]
        self.left      = self.progress[self.progress.index("/")+2:]
        self.launch    = driver.find_element_by_xpath(self.root+']/td[4]/div/ul/li[1]/a')            
        self.pause     = driver.find_element_by_xpath(self.root+']/td[4]/div/ul/li[2]/a')
        self.resume    = driver.find_element_by_xpath(self.root+']/td[4]/div/ul/li[3]/a')
        assert (self.name == self.project), "self.name != self.project"

    def __eq__(self, other):
        """Override the default equals behavior"""
        return self.completed == other.completed
    
    def __lt__(self, other):
        """Override the default less than behavior"""
        return self.completed < other.completed
    
    def __gt__(self, other):
        """Override the default greater than behavior"""
        return self.completed > other.completed

    def click_launch(self):
        if self.status == "Ready":
            self.launch.click()
            time.sleep(3)
            driver.find_element_by_xpath('//*[@id="page-top"]/div[7]/div/div[10]/button[1]').click()
            time.sleep(2)
        if self.status == "Stopped":
            self.resume.click()
            time.sleep(3)
            driver.find_element_by_xpath('//*[@id="alertify-ok"]').click()
            time.sleep(2)
        if self.status == "Completed":
            pass
            
    def click_pause(self):
        if self.status == "Running":
            self.pause.click()
            time.sleep(3)
            driver.find_element_by_xpath('//*[@id="alertify-ok"]').click()
            time.sleep(2)
        if self.status == "Completed":
            pass
        
    def recruit_completed(self, completed):
        #Increment completed count up and number remaining down
        self.completed += completed
        self.remaining -= completed

    def compare_progress(self, other):
        if self.completed == other.completed:
            print("  ", self.name, "is equal to", other.name)
            return "Equal"
        elif self.completed > other.completed:
            print("  ", self.name, "is ahead of", other.name, "by",
                str(self.completed - other.completed) + ":",
                self.completed, "to", other.completed)
            return other
        else:
            print("  ", other.name, "is ahead of", self.name, "by",
                str(other.completed - self.completed) + ":", 
                other.completed, "to", self.completed)
            return self

SyntaxError: invalid syntax (<ipython-input-102-f22d01694b06>, line 58)